In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
message = "Hello, GPT! This is my first message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! It's great to hear from you! How can I assist you today?


In [5]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [10]:
website_sample = Website("https://github.com")
print(website_sample.title)
print(website_sample.text)

GitHub · Build and ship software on a single, collaborative platform · GitHub
Skip to content
Navigation Menu
Toggle navigation
Sign in
Product
GitHub Copilot
Write better code with AI
GitHub Advanced Security
Find and fix vulnerabilities
Actions
Automate any workflow
Codespaces
Instant dev environments
Issues
Plan and track work
Code Review
Manage code changes
Discussions
Collaborate outside of code
Code Search
Find more, search less
Explore
Why GitHub
All features
Documentation
GitHub Skills
Blog
Solutions
By company size
Enterprises
Small and medium teams
Startups
Nonprofits
By use case
DevSecOps
DevOps
CI/CD
View all use cases
By industry
Healthcare
Financial services
Manufacturing
Government
View all industries
View all solutions
Resources
Topics
AI
DevOps
Security
Software Development
View all
Explore
Learning Pathways
Events & Webinars
Ebooks & Whitepapers
Customer Stories
Partners
Executive Insights
Open Source
GitHub Sponsors
Fund open source developers
The ReadME Project
GitH

In [11]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [12]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
                    please provide a short summary of this website in markdown. \
                    If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [13]:
print(user_prompt_for(website=website_sample))

You are looking at a website titled GitHub · Build and ship software on a single, collaborative platform · GitHub
The contents of this website is as follows;                     please provide a short summary of this website in markdown.                     If it includes news or announcements, then summarize these too.

Skip to content
Navigation Menu
Toggle navigation
Sign in
Product
GitHub Copilot
Write better code with AI
GitHub Advanced Security
Find and fix vulnerabilities
Actions
Automate any workflow
Codespaces
Instant dev environments
Issues
Plan and track work
Code Review
Manage code changes
Discussions
Collaborate outside of code
Code Search
Find more, search less
Explore
Why GitHub
All features
Documentation
GitHub Skills
Blog
Solutions
By company size
Enterprises
Small and medium teams
Startups
Nonprofits
By use case
DevSecOps
DevOps
CI/CD
View all use cases
By industry
Healthcare
Financial services
Manufacturing
Government
View all industries
View all solutions
Resources


In [14]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [15]:
messages_for(website=website_sample)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled GitHub · Build and ship software on a single, collaborative platform · GitHub\nThe contents of this website is as follows;                     please provide a short summary of this website in markdown.                     If it includes news or announcements, then summarize these too.\n\nSkip to content\nNavigation Menu\nToggle navigation\nSign in\nProduct\nGitHub Copilot\nWrite better code with AI\nGitHub Advanced Security\nFind and fix vulnerabilities\nActions\nAutomate any workflow\nCodespaces\nInstant dev environments\nIssues\nPlan and track work\nCode Review\nManage code changes\nDiscussions\nCollaborate outside of code\nCode Search\nFind more, search less\nExplore\nWhy GitHub\nAll features\nDocumentation\nGitHub Skills

In [16]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [17]:
summarize("https://github.com")

'# GitHub Overview\n\nGitHub is a collaborative platform designed for software development, integrating various tools to enhance productivity and streamline workflows. The website emphasizes the following key features:\n\n## Features\n\n- **GitHub Copilot**: An AI-powered coding assistant that helps developers write better code more efficiently. It integrates directly into workflows offering code completions and suggestions.\n- **Automated Workflows**: GitHub Actions enables CI/CD processes to optimize development workflows.\n- **Development Environments**: GitHub Codespaces provides instant cloud-based development setups.\n- **Security Measures**: GitHub Advanced Security employs AI to identify and fix vulnerabilities quickly.\n- **Project Management**: Tools for planning, tracking issues, and collaborating through GitHub Discussions and Issues.\n\n## Customer Stories\n\n- Duolingo reported a 25% increase in developer speed with GitHub Copilot.\n- Mercedes-Benz has standardized coding

In [18]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [19]:
display_summary("https://github.com")

# Summary of GitHub Website

GitHub is a comprehensive platform designed for building and shipping software collaboratively. It's widely recognized for its AI-powered tools, including GitHub Copilot, which enhances coding efficiency through features like code completion and chat support. The platform offers a range of functionalities such as:

- **GitHub Copilot**: AI-driven code assistance that helps developers write better code, improve productivity, and automate workflows.
- **GitHub Actions**: Tools for automating CI/CD workflows, optimizing development processes.
- **GitHub Codespaces**: Immediate cloud-based development environments that allow for instant project setup.
- **Advanced Security Features**: Tools to automatically find and fix vulnerabilities in code, enhancing application security.

GitHub serves various users, from startups to large enterprises, and integrates with over 17,000 applications, allowing for streamlined workflows and collaboration.

### Recent Highlights
- GitHub has become integral to many organizations, boosting developer speed and simplifying onboarding processes for new engineers.
- Case studies highlighting successful integrations of GitHub's tools showcase significant reductions in development time and improved security measures across industries.

Overall, GitHub is positioned as a key platform for developers seeking efficiency, security, and collaboration in their software development projects.